In [1]:
import json
import sqlite3
import requests

DB_PATH = "mlb_data.db"

def get_conn(db_path=DB_PATH):
    conn = sqlite3.connect(db_path)
    # sensible SQLite pragmas for local dev
    conn.execute("PRAGMA journal_mode = WAL;")
    conn.execute("PRAGMA foreign_keys = ON;")
    return conn

conn = get_conn()
cur = conn.cursor()
print("Connected to", DB_PATH)


Connected to mlb_data.db


In [2]:
cur.execute("""
CREATE TABLE IF NOT EXISTS teams_bronze (
    team_id INTEGER PRIMARY KEY,
    json_data TEXT NOT NULL,
    loaded_utc TEXT NOT NULL DEFAULT (datetime('now'))
)
""")
conn.commit()
print("teams_bronze ready")


teams_bronze ready


In [3]:
url = "https://statsapi.mlb.com/api/v1/teams"
params = {"sportId": 1}  # 1 = MLB
resp = requests.get(url, params=params, timeout=30)
resp.raise_for_status()
data = resp.json()

teams = data.get("teams", [])
len(teams), teams[0]["name"]


(30, 'Athletics')

In [4]:
rows = [(t["id"], json.dumps(t, ensure_ascii=False)) for t in teams]

cur.executemany("""
INSERT INTO teams_bronze (team_id, json_data, loaded_utc)
VALUES (?, ?, datetime('now'))
ON CONFLICT(team_id) DO UPDATE SET
    json_data = excluded.json_data,
    loaded_utc = excluded.loaded_utc
""", rows)
conn.commit()
print("Upserted", len(rows), "rows into teams_bronze")

Upserted 30 rows into teams_bronze


In [5]:
cur.execute("SELECT COUNT(*) FROM teams_bronze")
count = cur.fetchone()[0]
cur.execute("SELECT team_id, SUBSTR(json_data, 1, 120) FROM teams_bronze ORDER BY team_id LIMIT 3")
sample = cur.fetchall()
count, sample

(30,
 [(108,
   '{"springLeague": {"id": 114, "name": "Cactus League", "link": "/api/v1/league/114", "abbreviation": "CL"}, "allStarStatu'),
  (109,
   '{"springLeague": {"id": 114, "name": "Cactus League", "link": "/api/v1/league/114", "abbreviation": "CL"}, "allStarStatu'),
  (110,
   '{"springLeague": {"id": 115, "name": "Grapefruit League", "link": "/api/v1/league/115", "abbreviation": "GL"}, "allStarS')])

In [6]:
import pandas as pd
import json

# pull all raw JSON rows from bronze
rows = []
for (team_id, json_text) in cur.execute("SELECT team_id, json_data FROM teams_bronze"):
    rows.append(json.loads(json_text))

# normalize nested structures into columns (keep everything)
df_stage = pd.json_normalize(rows, sep="_")

# optional: basic hygiene for numeric fields you’ll likely use later
for col in ("season", "firstYearOfPlay"):
    if col in df_stage.columns:
        df_stage[col] = pd.to_numeric(df_stage[col], errors="coerce").fillna(0).astype(int)

# write stage table (replace each run — it's a scratch/working table)
df_stage.to_sql("teams_stage", conn, if_exists="replace", index=False)

df_stage.shape, df_stage.columns[:12].tolist()



((30, 33),
 ['allStarStatus',
  'id',
  'name',
  'link',
  'season',
  'teamCode',
  'fileCode',
  'abbreviation',
  'teamName',
  'locationName',
  'firstYearOfPlay',
  'shortName'])

In [7]:
cur.execute("SELECT COUNT(*) FROM teams_stage")
count = cur.fetchone()[0]
cur.execute("""
  SELECT id, name, abbreviation, teamCode, league_name, division_name, venue_name, active
  FROM teams_stage ORDER BY id LIMIT 5
""")
count, cur.fetchall()


(30,
 [(108,
   'Los Angeles Angels',
   'LAA',
   'ana',
   'American League',
   'American League West',
   'Angel Stadium',
   1),
  (109,
   'Arizona Diamondbacks',
   'AZ',
   'ari',
   'National League',
   'National League West',
   'Chase Field',
   1),
  (110,
   'Baltimore Orioles',
   'BAL',
   'bal',
   'American League',
   'American League East',
   'Oriole Park at Camden Yards',
   1),
  (111,
   'Boston Red Sox',
   'BOS',
   'bos',
   'American League',
   'American League East',
   'Fenway Park',
   1),
  (112,
   'Chicago Cubs',
   'CHC',
   'chn',
   'National League',
   'National League Central',
   'Wrigley Field',
   1)])

In [8]:
cur.execute("""
CREATE TABLE IF NOT EXISTS teams_silver (
    team_id INTEGER PRIMARY KEY,
    team_name TEXT,
    abbrev TEXT,
    team_code TEXT,
    franchise_name TEXT,
    club_name TEXT,
    league TEXT,
    division TEXT,
    location TEXT,
    season INTEGER,
    first_year_of_play INTEGER,
    venue_name TEXT,
    active INTEGER
)
""")
conn.commit()
print("teams_silver ready")


teams_silver ready


In [9]:
cur.execute("""
INSERT OR REPLACE INTO teams_silver (
  team_id, team_name, abbrev, team_code, franchise_name, club_name,
  league, division, location, season, first_year_of_play, venue_name, active
)
SELECT
  id,
  name,
  abbreviation,
  teamCode,
  franchiseName,
  teamName,
  league_name,
  division_name,
  locationName,
  season,
  firstYearOfPlay,
  venue_name,
  CASE WHEN active THEN 1 ELSE 0 END
FROM teams_stage
""")
conn.commit()
cur.rowcount  # rows written to silver


30

In [10]:
cur.execute("""
SELECT team_id, abbrev, club_name, league, division, venue_name, active
FROM teams_silver
ORDER BY abbrev
LIMIT 10
""")
cur.fetchall()


[(133,
  'ATH',
  'Athletics',
  'American League',
  'American League West',
  'Sutter Health Park',
  1),
 (144,
  'ATL',
  'Braves',
  'National League',
  'National League East',
  'Truist Park',
  1),
 (109,
  'AZ',
  'D-backs',
  'National League',
  'National League West',
  'Chase Field',
  1),
 (110,
  'BAL',
  'Orioles',
  'American League',
  'American League East',
  'Oriole Park at Camden Yards',
  1),
 (111,
  'BOS',
  'Red Sox',
  'American League',
  'American League East',
  'Fenway Park',
  1),
 (112,
  'CHC',
  'Cubs',
  'National League',
  'National League Central',
  'Wrigley Field',
  1),
 (113,
  'CIN',
  'Reds',
  'National League',
  'National League Central',
  'Great American Ball Park',
  1),
 (114,
  'CLE',
  'Guardians',
  'American League',
  'American League Central',
  'Progressive Field',
  1),
 (115,
  'COL',
  'Rockies',
  'National League',
  'National League West',
  'Coors Field',
  1),
 (145,
  'CWS',
  'White Sox',
  'American League',
  'Ameri

In [12]:
cur.close()
conn.close()
print("Closed DB.")



Closed DB.
